In [1]:
import os
import sys
import gc
import ctypes
import numpy as np
import pandas as pd
import platform
import logging
from datetime import datetime, timedelta
from pathlib import Path
import shutil
import zipfile
import duckdb 
import warnings
import fastparquet
from tqdm import tqdm 
from typing import List, Optional, Union
import psutil
import time # For timing the execution

warnings.filterwarnings('ignore')


# Optional: adjust pandas display for debugging; you can comment these out
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)


logging.basicConfig(level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S')

In [2]:
mapping_file = "../../../data_points/Naarni VehicleID_RegNo_links - Vehicle_mapping.csv"
try:
    df_mapping = pd.read_csv(mapping_file)
except FileNotFoundError:
    print(f"Error: Mapping file '{mapping_file}' not found. Cannot enrich data.")
    # Create an empty mapping table to allow the rest of the script to run without crashing
    df_mapping = pd.DataFrame(columns=["id", "reg_num", "customer", "model"])
else:
    df_mapping = df_mapping.rename(columns={
        "Device No.": "id",
        "Registration No": "reg_num",
        "Customer": "customer",
        "Model": "model"
    })
    # Ensure the merge key ('id') is a string to match the chunks
    if "id" in df_mapping.columns:
        df_mapping["id"] = df_mapping["id"].astype(str)
        df_mapping = df_mapping[["id", "reg_num", "customer", "model"]]
    else:
        print("Warning: 'Device No.' column not found in mapping file.")
        df_mapping = pd.DataFrame(columns=["id", "reg_num", "customer", "model"])

print(f"Loaded mapping table with {len(df_mapping)} entries.")
# df_mapping is now ready to be passed into the processing function

df_mapping.head()

Loaded mapping table with 27 entries.


,id,reg_num,customer,model
0,11,AP39WF8593,FB Guntur,12.5
1,9,AP39WG0252,FB Guntur,12.5
2,7,AP39WF8589,FB Guntur,12.5
3,13,AP39WF8584,FB Guntur,12.5
4,14,AP39WG0271,FB Guntur,12.5


In [3]:
CORE_COLS = [
    "id", "timestamp", "dt",
    "vehiclereadycondition", "gun_connection_status", "ignitionstatus",
    "vehicle_speed_vcu", "gear_position",
    "bat_soc", "soh", "total_battery_current",
    "pack1_cellmax_temperature", "pack1_cell_min_temperature",
    "pack1_maxtemperature_cell_number", "pack1_celltemperature_cellnumber",
    "bat_voltage", "cellmax_voltagecellnumber", "cellminvoltagecellnumber", 
    "cell_min_voltage","cell_max_voltage",
]

In [4]:
def free_mem():
    """Try to return freed memory back to the OS (no-op on some platforms)."""
    try:
        libc = ctypes.CDLL(None)
        if hasattr(libc, "malloc_trim"):
            libc.malloc_trim(0)
    except Exception:
        pass

In [5]:
def rename_battery_temp_columns(df: pd.DataFrame) -> pd.DataFrame:
    # Uses df.rename(inplace=False), creating one copy, which is fine for chunks
    rename_map = {
        "pack1_cellmax_temperature": "batt_maxtemp",
        "pack1_cell_min_temperature": "batt_mintemp",
        "pack1_maxtemperature_cell_number":"batt_maxtemp_tc", 
        "pack1_celltemperature_cellnumber":"batt_mintemp_tc",
        "cell_max_voltage":"batt_maxvolt",
        "cellmax_voltagecellnumber":"batt_maxvolt_cell",
        "cell_min_voltage":"batt_minvolt",
        "cellminvoltagecellnumber":"batt_minvolt_cell", 
    }
    existing = {k: v for k, v in rename_map.items() if k in df.columns}
    if not existing:
        return df
    return df.rename(columns=existing)

In [6]:
def impute_missing_values(df: pd.DataFrame) -> pd.DataFrame:
    """
    Cleans invalid sensor values and imputes missing values using
    forward-fill (with limit) followed by backfill.
    """
    df = df.sort_values(["id", "timestamp"]).copy()

    # ---------------------------------------------------------------
    # 1. SANITISATION: Convert physically impossible values to NaN
    # ---------------------------------------------------------------

    # --- 1a. Temperature sanitisation: -40°C and anything < -10°C is invalid ---
    temp_cols = ["batt_maxtemp", "batt_mintemp"]
    for col in temp_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")
            df.loc[df[col] < -10, col] = pd.NA

    # --- 1b. Pack voltages: cannot be 0V or negative ---
    volt_cols = ["batt_maxvolt", "batt_minvolt", "bat_voltage"]
    for col in volt_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")
            df.loc[df[col] <= 0, col] = pd.NA

    # --- 1c. Thermocouple sensor IDs cannot be 0 or invalid ---
    tc_cols = ["batt_maxtemp_tc", "batt_mintemp_tc"]
    for col in tc_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")
            df.loc[(df[col] < 1) | (df[col] > 108), col] = pd.NA

    # --- 1d. Cell IDs cannot be 0 or invalid ---
    cell_cols = ["batt_maxvolt_cell", "batt_minvolt_cell"]
    for col in cell_cols:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")
            df.loc[(df[col] < 1) | (df[col] > 576), col] = pd.NA

    # --- 1e. CURRENT SANITISATION: Clip current values outside [-2500, 2500] ---
    current_col = "total_battery_current"
    if current_col in df.columns:
        df[current_col] = pd.to_numeric(df[current_col], errors="coerce")
        # Set values > 2500 or < -2500 to NaN
        df.loc[abs(df[current_col]) > 2500, current_col] = pd.NA

    current_col = "total_battery_current"

    # 1. Clip positive outliers (Only the current column is affected)
    df.loc[df[current_col] > 2500, current_col] = pd.NA

    # 2. Clip negative outliers (Only the current column is affected)
    df.loc[df[current_col] < -2500, current_col] = pd.NA

    # ---------------------------------------------------------------
    # 2. GROUPWISE FORWARD-FILL + BACKFILL IMPUTATION
    # ---------------------------------------------------------------
    impute_config = [
        ("batt_maxtemp", 60),
        ("batt_mintemp", 60),
        ("batt_maxtemp_tc", 60),
        ("batt_mintemp_tc", 60),

        ("batt_maxvolt", 30),
        ("batt_minvolt", 30),
        ("batt_maxvolt_cell", 30),
        ("batt_minvolt_cell", 30),

        ("bat_voltage", 20),
        ("bat_soc", 300),
        ("soh", 300),
    ]

    for vid, grp in df.groupby("id", sort=False):
        idx = grp.index

        # Apply ffill(limit) then bfill() for each column
        for col, limit in impute_config:
            if col in df.columns:
                cleaned = grp[col].ffill(limit=limit).bfill()   # key fix here
                df.loc[idx, col] = cleaned

        # Total current interpolation (smooth & symmetric)
        if "total_battery_current" in df.columns:
            df.loc[idx, "total_battery_current"] = grp["total_battery_current"].interpolate(
                limit=10, limit_direction="both"
            )

        # Ready condition + GCS should never remain missing
        for col in ["vehiclereadycondition", "gun_connection_status"]:
            if col in df.columns:
                df.loc[idx, col] = grp[col].ffill().bfill()

    return df


In [ ]:
import pandas as pd
import numpy as np

def prepare_df_with_state(df: pd.DataFrame, df_mapping: pd.DataFrame) -> pd.DataFrame:
    # CRITICAL: This line creates the necessary copy, but now it's only a small chunk!
    out = df.copy() 
    
    # 1. Merge Vehicle Information
    out["id"] = out["id"].astype(str)  # Ensure merge key type consistency
    out = out.merge(
        df_mapping, 
        on="id", 
        how="left", 
        validate="m:1"
    )

    # --- Fill Missing Mapping Information ---
    out["reg_num"] = out["reg_num"].mask(
        out["reg_num"].isna(),
        "REGNUM_" + out["id"].astype(str)
    )
    out["customer"] = out["customer"].mask(
        out["customer"].isna(),
        "CUST_" + out["id"].astype(str)
    )
    out["model"] = out["model"].mask(
        out["model"].isna(),
        "MDL_" + out["id"].astype(str)
    )

    # Ensure string dtype for PyArrow
    out["reg_num"] = out["reg_num"].astype(str)
    out["customer"] = out["customer"].astype(str)
    out["model"] = out["model"].astype(str)

    # 2. Basic Cleanup & Sorting (to IST)
    # The timestamp processing remains the same for consistency and sorting
    out["timestamp"] = (
        out["timestamp"]
        .dt.tz_localize("UTC")
        .dt.tz_convert("Asia/Kolkata")
    )
    out["timestamp"] = pd.to_datetime(out["timestamp"], errors="coerce")
    out = (
        out.dropna(subset=["timestamp"])
           .sort_values(["id", "timestamp"])
           .reset_index(drop=True)
    )
    
    # 3. Mode calculation (Includes new alt_mode)
    
    # Helper Series for Gun Connection Status (GCS)
    gcs_raw = out["gun_connection_status"]
    gcs_num = pd.to_numeric(gcs_raw, errors="coerce")
    gcs_str = gcs_raw.astype(str).str.strip().str.lower()
    gun_connected = (gcs_num == 1) | gcs_str.isin(
        {"1", "true", "yes", "y", "connected", "on"}
    )
    gun_connected = gun_connected.fillna(False)
    
    # Helper Series for Vehicle Ready Condition (VRC)
    if "vehiclereadycondition" in out.columns:
        vrc_raw = out["vehiclereadycondition"]
        # Assuming VRC is numeric (1 or 0) but handling potential string/bool
        vrc_num = pd.to_numeric(vrc_raw, errors="coerce")
        vrc_str = vrc_raw.astype(str).str.strip().str.lower()
        vehicle_ready = (vrc_num == 1) | vrc_str.isin(
            {"1", "true", "yes", "y", "ready", "on"}
        )
        # Default to not ready if missing, to prevent false DRIVING
        vehicle_ready = vehicle_ready.fillna(False) 
    else:
        # If column is missing, assume not ready
        vehicle_ready = pd.Series([False] * len(out), index=out.index)
    
    # 3a. Original 'mode' calculation (for backward compatibility)
    out["mode"] = np.where(gun_connected, "CHARGING", "DISCHARGING")
    
    # 3b. NEW 'alt_mode' calculation (CHARGING, DRIVING, IDLE)
    
    # The alt_mode is built using three mutually exclusive conditions:
    # 1. CHARGING: GUN_CONNECTION_STATUS == 1 (The VRC/Ignition status doesn't matter here)
    # 2. DRIVING: GUN_CONNECTION_STATUS == 0 AND VEHICLE_READY_CONDITION == 1
    # 3. IDLE: GUN_CONNECTION_STATUS == 0 AND VEHICLE_READY_CONDITION == 0
    
    conditions = [
        gun_connected,
        (~gun_connected) & vehicle_ready
    ]
    
    choices = [
        "CHARGING",
        "DRIVING"
    ]
    
    # The default value (else) for np.select will be IDLE, 
    # which covers the condition: (~gun_connected) & (~vehicle_ready)
    out["alt_mode"] = np.select(
        conditions, 
        choices, 
        default="IDLE"
    )

    # 4. Delta calculations
    for col in ["batt_maxtemp", "batt_mintemp", "batt_maxvolt", "batt_minvolt"]:
        if col in out.columns:
            out[col] = pd.to_numeric(out[col], errors="coerce")
            
    out["batt_temp_delta"] = out["batt_maxtemp"] - out["batt_mintemp"]
    out["volt_delta_mv"] = np.round(
        (out["batt_maxvolt"] - out["batt_minvolt"]) * 1000.0, 0
    )
    out["dt_sec"] = (
        out.groupby("id")["timestamp"].diff().dt.total_seconds().fillna(0)
    )
    out.loc[out["dt_sec"] > 3, "dt_sec"] = 0

    # --- NEW: extract date from timestamp ---
    out["date_val"] = out["timestamp"].dt.floor("D")

    # --- NEW: pack_id_max & pack_id_min from thermocouple indices (multiples of 9) ---

    # Initialize as nullable integer columns
    out["pack_id_max"] = pd.Series([pd.NA] * len(out), dtype="Int64")
    out["pack_id_min"] = pd.Series([pd.NA] * len(out), dtype="Int64")

    # batt_maxtemp_tc -> pack_id_max
    if "batt_maxtemp_tc" in out.columns:
        tc_max = pd.to_numeric(out["batt_maxtemp_tc"], errors="coerce")
        mask_max = tc_max.notna() & (tc_max > 0)
        out.loc[mask_max, "pack_id_max"] = (
            ((tc_max[mask_max] - 1) // 9) + 1
        ).astype("Int64")

    # batt_mintemp_tc -> pack_id_min
    if "batt_mintemp_tc" in out.columns:
        tc_min = pd.to_numeric(out["batt_mintemp_tc"], errors="coerce")
        mask_min = tc_min.notna() & (tc_min > 0)
        out.loc[mask_min, "pack_id_min"] = (
            ((tc_min[mask_min] - 1) // 9) + 1
        ).astype("Int64")

    # 5. Feature Buckets (Binning)
    temp_bins = [-np.inf, 28, 32, 35, 40, np.inf]
    temp_labels = ["<28", "28–32", "32–35", "35–40", ">40"]

    delta_bins = [-np.inf, 2, 5, 8, np.inf]
    delta_labels = ["<2", "2–5", "5–8", ">8"]

    volt_bins = [0, 10, 20, 30, np.inf]
    volt_labels = ["0–10", "10–20", "20–30", ">30"]

    soc_bins = [0, 10, 20, 30, 40, 50, 60, 70, 80, 90, np.inf]
    soc_labels = ["0–10", "10–20", "20–30", "30-40", "40-50", "50-60", "60-70","70-80", "80-90","90-100"]    
    
    out["maxtemp_bucket"] = pd.cut(
        out["batt_maxtemp"], bins=temp_bins, labels=temp_labels
    )
    out["temp_delta_bucket"] = pd.cut(
        out["batt_temp_delta"], bins=delta_bins, labels=delta_labels
    )
    out["volt_delta_bucket"] = pd.cut(
        out["volt_delta_mv"], bins=volt_bins, labels=volt_labels
    )
    out["soc_band_bucket"] = pd.cut(
        out["bat_soc"], bins=soc_bins, labels=soc_labels
    )    

    # 6. Column selection (alt_mode is added to the list)
    cols_keep = [
        "id", "reg_num", "customer", "model",
        "timestamp",'date_val', "dt_sec", "mode", "alt_mode",
        "vehiclereadycondition", "gun_connection_status",
        "batt_maxtemp", "batt_mintemp", "batt_temp_delta",
        "maxtemp_bucket", "temp_delta_bucket",
        "batt_maxvolt", "batt_minvolt", "volt_delta_mv",
        "volt_delta_bucket",
        "batt_maxtemp_tc", "batt_mintemp_tc",
        "pack_id_max", "pack_id_min",
        "batt_maxvolt_cell", "batt_minvolt_cell",
        "bat_voltage", "total_battery_current",
        "bat_soc","soc_band_bucket", "soh",
    ]
    cols_keep = [c for c in cols_keep if c in out.columns]
    out = out[cols_keep]

    return out

In [8]:
# --- DUCKDB CHUNK GENERATOR (Fixed) ---

def duckdb_chunk_generator(conn, sql_query, chunk_size):
    """Generates Pandas DataFrames in chunks directly from DuckDB cursor."""
    cursor = conn.cursor() 
    cursor.execute(sql_query)
    
    while True:
        # Uses the corrected method name: fetch_df_chunk
        chunk = cursor.fetch_df_chunk(chunk_size) 
        if chunk is None or chunk.empty:
            break
        yield chunk

# --- ROBUST FILE EXTRACTION (Fixed from OSErrors) ---

def extract_files_to_disk(zip_path, output_dir):
    """Cleans directory and extracts all Parquet files from ZIP."""
    if output_dir.exists():
        logging.info(f"🧹 Clearing existing directory: {output_dir.resolve()}")
        # Robust cleanup to avoid OS/lock issues
        try:
            shutil.rmtree(output_dir)
        except OSError:
             for item in output_dir.iterdir():
                if item.is_dir():
                    shutil.rmtree(item)
                else:
                    os.remove(item) 
             os.rmdir(output_dir)

    output_dir.mkdir(parents=True)
        
    logging.info("🔄 Extracting ALL Parquet files from ZIP to disk...")
    try:
        with zipfile.ZipFile(zip_path, "r") as z:
            all_files_to_extract = [f for f in z.namelist() if f.endswith(".parquet")]
            logging.info(f"🔎 Found {len(all_files_to_extract)} total Parquet files in archive.")
            for filename in all_files_to_extract:
                z.extract(filename, path=output_dir)
            return len(all_files_to_extract)
    except FileNotFoundError:
        raise FileNotFoundError(f"❌ ZIP file not found at: {zip_path}") from None

def setup_duckdb_query(output_dir, utc_start, utc_end, core_cols):
    """Sets up DuckDB connection and SQL query."""
    parquet_glob_path = str(output_dir.joinpath("**/*.parquet"))
    # Only select the columns you need for Stage 1 processing
    column_list = ", ".join([f'"{c}"' for c in core_cols])
    
    # CRITICAL: Predicate Pushdown filter on the internal 'timestamp' column
    sql_query = f"""
        SELECT {column_list}
        FROM read_parquet('{parquet_glob_path}')
        WHERE 
            "timestamp" >= '{utc_start.isoformat()}' AND 
            "timestamp" < '{utc_end.isoformat()}'
    """
    return duckdb.connect(), sql_query

In [9]:
def run_stage1_data_setup(analysis_start_date_str: str, 
                          analysis_end_date_str: str, 
                          zip_path: Path, 
                          extraction_dir: Path,
                          force_extraction: bool = False) -> tuple[datetime, datetime, int]:
    """
    Handles date range setup, IST-to-UTC conversion, file extraction, 
    and checks if data is available for processing.
    
    Args:
        analysis_start_date_str: Start date in YYYY-MM-DD format.
        analysis_end_date_str: End date in YYYY-MM-DD format.
        zip_path: Path to the source ZIP file.
        extraction_dir: Target directory for extracted Parquet files.
        force_extraction: If True, always clean and re-extract files. 
                          If False, skips extraction if the directory exists.
    
    Returns: (utc_start, utc_end, file_count)
    """
    
    # 1. Date Parsing and UTC Conversion (Assuming +5:30 IST offset)
    target_date = datetime.strptime(analysis_start_date_str, "%Y-%m-%d").date()
    ist_start = datetime.combine(target_date, datetime.min.time())
    
    end_date_obj = datetime.strptime(analysis_end_date_str, "%Y-%m-%d").date()
    ist_end = datetime.combine(end_date_obj, datetime.min.time()) + timedelta(days=1)
    
    utc_start = ist_start - timedelta(hours=5, minutes=30)
    utc_end = ist_end - timedelta(hours=5, minutes=30)
    
    logging.info(f"🔍 Analysis window (UTC): {utc_start} → {utc_end}")

    # 2. FILE EXTRACTION CONTROL
    file_count = 0
    
    if extraction_dir.exists() and not force_extraction:
        logging.info("♻️ Skipping file extraction: Directory exists and force_extraction=False.")
        # Recursively count all .parquet files in the existing directory
        file_count = len(list(extraction_dir.rglob('*.parquet')))
        if file_count > 0:
             logging.info(f"✅ Found {file_count} existing files. Proceeding to DuckDB loading.")
        
    else:
        # If directory doesn't exist, or force_extraction is True, run the full extraction.
        logging.info("🔄 Running full extraction (Cleanup + Extract)...")
        # This relies on the robust `extract_files_to_disk` function
        file_count = extract_files_to_disk(zip_path, extraction_dir)
        
    # 3. Validation Check
    if file_count == 0:
        logging.warning("🛑 Skipping analysis: No files were found.")
        sys.exit() # Exit the script cleanly if no files were found

    return utc_start, utc_end, file_count

In [10]:
# --- CONFIGURATION (Ensure these are defined at the top of your script) ---
ZIP_FILE_PATH = "../../../data_points/naarni75_cpoall.zip" 
EXTRACTION_DIR = Path("../../../data_points/extracted_parts/cpo_all") 
# --------------------------------------------------------------------------

# --- Set 75-Day Date Range (Using your target dates) ---
analysis_start_date_str = "2025-09-01" 
# NOTE: Using 2025-11-14 since 75 days starts on 2025-09-01 and ends on 2025-11-14.
# Using 2025-11-15 will include the start of the 76th day (if data exists).
analysis_end_date_str = "2025-11-15"   
# --------------------------------------------------------

# NEW STAGE 1 EXECUTION:
utc_start, utc_end, file_count = run_stage1_data_setup(
    analysis_start_date_str=analysis_start_date_str,
    analysis_end_date_str=analysis_end_date_str,
    zip_path=ZIP_FILE_PATH,
    extraction_dir=EXTRACTION_DIR,
    force_extraction = False)

2025-11-26 18:18:22 - INFO - 🔍 Analysis window (UTC): 2025-08-31 18:30:00 → 2025-11-15 18:30:00
2025-11-26 18:18:22 - INFO - ♻️ Skipping file extraction: Directory exists and force_extraction=False.
2025-11-26 18:18:22 - INFO - ✅ Found 474 existing files. Proceeding to DuckDB loading.


In [11]:
# 2. SETUP DUCKDB QUERY
conn, sql_query = setup_duckdb_query(EXTRACTION_DIR, utc_start, utc_end, CORE_COLS)

# A quick DuckDB query to get the distinct IDs from the 75-day filtered dataset
get_ids_query = f"""
    SELECT DISTINCT id 
    FROM ({sql_query})
"""
# Fetch the list of IDs (this is a very small amount of data)
vehicle_ids = conn.execute(get_ids_query).fetchdf()["id"].astype(str).tolist()
# vehicle_ids = ['3','16','18','19','32','42','6','7','9','11','12','13','14','15','20','25','27','28','29','30','31','33','35','41','46']

logging.info(f"✅ Found {len(vehicle_ids)} unique vehicle IDs in the dataset.")
print(sorted(vehicle_ids))

2025-11-26 18:18:23 - INFO - ✅ Found 29 unique vehicle IDs in the dataset.


['10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '22', '24', '25', '26', '27', '28', '29', '3', '30', '31', '32', '33', '41', '42', '46', '6', '7', '9']


In [12]:
# NOTE: The functions 'duckdb_chunk_generator', 'rename_battery_temp_columns', 
# 'impute_missing_values', 'prepare_df_with_state', and 'free_mem' 
# MUST be defined in your Jupyter Notebook environment before calling this.

def process_and_save_data(
    conn: duckdb.DuckDBPyConnection, 
    sql_query: str, 
    chunk_size: int, 
    parquet_feather_path: str,
    vehicle_ids: List[str],
    df_mapping: pd.DataFrame,
    extract_data: bool = False,
    chunk_log_path: Optional[Union[str, Path]] = None,
) -> int:
    """
    Executes the memory-safe chunked processing loop for Stage 1.
    
    MODIFIED:
      - Pre-calculates total rows for a 0-100% progress bar.
      - Suppresses INFO logs during the loop (like run_multi_day_session_analysis).
      - Updates progress based on rows processed.
    """

    output_path = Path(parquet_feather_path)

    # -------------------------------------------------------
    # SKIP IF FILE EXISTS
    # -------------------------------------------------------
    if not extract_data and output_path.exists():
        # We use a separate connection to check the existing file
        conn_count = duckdb.connect()
        try:
            total_rows = conn_count.execute(
                f"SELECT count(*) FROM '{parquet_feather_path}'"
            ).fetchone()[0]
            logging.info(f"✅ Skipping: File already exists with {total_rows:,} rows.")
            return total_rows
        finally:
            conn_count.close()

    # -------------------------------------------------------
    # INIT & PRE-CALCULATION
    # -------------------------------------------------------
    logging.info(f"🧠 Preparing data stream...")
    
    # 1. Get Total Rows for the Progress Bar (Enables 0-100% visualization)
    #    We wrap the user query in a count to get the total volume.
    try:
        total_input_rows = conn.execute(f"SELECT COUNT(*) FROM ({sql_query})").fetchone()[0]
        logging.info(f"📊 Total rows to process: {total_input_rows:,}")
    except Exception as e:
        logging.warning(f"Could not determine total row count: {e}. Progress bar will be indefinite.")
        total_input_rows = None

    logging.info(f"💾 Output path: {parquet_feather_path}")

    first_chunk = True
    total_processed_rows = 0
    chunk_index = 0

    # psutil process handle (for RAM)
    process = psutil.Process(os.getpid())
    psutil.cpu_percent(interval=None) # Prime cpu

    # Chunk timing log setup
    log_file = None
    if chunk_log_path is not None:
        log_file = Path(chunk_log_path)
        if not log_file.exists():
            with log_file.open("w") as f:
                f.write("timestamp,chunk_idx,chunk_rows,total_rows,duration_sec,rows_per_sec,cpu_pct,ram_mb\n")

    # -------------------------------------------------------
    # LOGGING SUPPRESSION
    # -------------------------------------------------------
    # Save current level and set to WARNING to hide internal INFO logs during the bar update
    current_log_level = logging.getLogger().level
    logging.getLogger().setLevel(logging.WARNING)

    try:
        # -------------------------------------------------------
        # PROGRESS BAR
        # -------------------------------------------------------
        # We use 'total=total_input_rows' to get the percentage bar
        progress = tqdm(
            total=total_input_rows,
            desc="Processing Data",
            unit="row",
            mininterval=0.2,
            dynamic_ncols=True,
            colour="cyan" # Keeps the cyan color you had, distinct from the other function
        )

        # -------------------------------------------------------
        # MAIN LOOP: STREAM CHUNKS
        # -------------------------------------------------------
        for chunk in duckdb_chunk_generator(conn, sql_query, chunk_size):
            t0 = time.perf_counter()
            chunk_index += 1
            
            # We capture the raw size here to update the progress bar accurately 
            # relative to the input stream
            raw_chunk_len = len(chunk) 

            df_chunk = chunk

            # --- PREP 1: Rename ---
            df_chunk = rename_battery_temp_columns(df_chunk)

            # --- PREP 2: Filter & Cast ---
            if "id" in df_chunk.columns:
                df_chunk["id"] = df_chunk["id"].astype(str)
                if vehicle_ids:
                    df_chunk = df_chunk[df_chunk["id"].isin(vehicle_ids)]
                df_chunk = df_chunk.convert_dtypes()

            # --- PREP 3: Time & Impute ---
            df_chunk["timestamp"] = pd.to_datetime(df_chunk["timestamp"], errors="coerce")
            df_chunk = impute_missing_values(df_chunk)

            # --- PREP 4: State Prep ---
            df_chunk_state = prepare_df_with_state(df_chunk, df_mapping)

            if df_chunk_state.empty:
                # Even if empty, we processed 'raw_chunk_len' rows from source
                progress.update(raw_chunk_len) 
                progress.set_postfix_str("Skipped empty chunk")
                del df_chunk, df_chunk_state
                gc.collect()
                free_mem()
                continue

            rows_saved_this_chunk = len(df_chunk_state)
            total_processed_rows += rows_saved_this_chunk

            # --- SAVE CHUNK ---
            if first_chunk:
                df_chunk_state.to_parquet(parquet_feather_path, compression="zstd", index=False)
                first_chunk = False
            else:
                fastparquet.write(parquet_feather_path, df_chunk_state, compression="zstd", write_index=False, append=True)

            # --- METRICS ---
            t1 = time.perf_counter()
            duration = t1 - t0
            rows_per_sec = rows_saved_this_chunk / duration if duration > 0 else 0
            cpu_pct = psutil.cpu_percent(interval=None)
            ram_mb = process.memory_info().rss / (1024 * 1024)

            # --- UPDATE PROGRESS ---
            # Update the bar by the amount of data read from SQL (to match total_input_rows)
            progress.update(raw_chunk_len)
            
            # Update the text stats with system metrics
            progress.set_postfix(
                saved=f"{total_processed_rows:,}", # Rows actually saved to disk
                cpu=f"{cpu_pct:4.1f}%",
                ram=f"{ram_mb:6.1f}MB",
                speed=f"{rows_per_sec:8.1f} r/s"
            )

            # --- FILE LOGGING ---
            if log_file is not None:
                with log_file.open("a") as f:
                    f.write(
                        f"{datetime.now().isoformat()},{chunk_index},"
                        f"{rows_saved_this_chunk},{total_processed_rows},"
                        f"{duration:.3f},{rows_per_sec:.1f},"
                        f"{cpu_pct:.1f},{ram_mb:.1f}\n"
                    )

            # --- CLEANUP ---
            del df_chunk, df_chunk_state
            gc.collect()
            free_mem()

    finally:
        # -------------------------------------------------------
        # RESTORE LOGGING & CLOSE
        # -------------------------------------------------------
        if 'progress' in locals():
            progress.close()
        
        # Restore the original logging level so subsequent code isn't silenced
        logging.getLogger().setLevel(current_log_level)
        
        conn.close()

    logging.info(f"✅ Finished processing. Total rows saved: {total_processed_rows:,}")
    return total_processed_rows

In [ ]:
# --- ASSUMING run_stage1_data_setup WAS CALLED AND RETURNED utc_start, utc_end ---
# Example configuration that needs to be available:
# EXTRACTION_DIR = Path("../extracted_parts") 
# CORE_COLS = [...]
# CRITICAL FIX: Drastically reduced chunk size to prevent memory spike
CHUNK_SIZE = 1_000 # Process 50,000 rows max at any time


# 1. Setup DuckDB Query (as shown previously)
conn, sql_query = setup_duckdb_query(EXTRACTION_DIR, utc_start, utc_end, CORE_COLS)

# 2. Define Inputs
# output_feather_file = "df_with_state_30days.feather"
output_parquet_file = "../df_with_state.parquet"

# 3. Run the memory-safe processing loop
total_rows = process_and_save_data(
    conn=conn,
    sql_query=sql_query,
    chunk_size=CHUNK_SIZE,
    parquet_feather_path=output_parquet_file,
    vehicle_ids=vehicle_ids,
    df_mapping=df_mapping,
    extract_data=True
)

logging.info(f"🎉 Final DataFrame saved. Total rows processed: {total_rows:,}")
logging.info("✅ Data Processing (Stage 1) complete. Feather file ready for analysis.")

2025-11-26 18:18:23 - INFO - ✅ Skipping: File already exists with 52,888,998 rows.
2025-11-26 18:18:23 - INFO - 🎉 Final DataFrame saved. Total rows processed: 52,888,998
2025-11-26 18:18:23 - INFO - ✅ Data Processing (Stage 1) complete. Feather file ready for analysis.


In [14]:
def read_parquet_subset(parquet_path: str, start_dt: datetime, end_dt: datetime) -> pd.DataFrame:
    """
    Reads a subset of the processed Feather file using a date filter
    applied directly by DuckDB (predicate pushdown).
    
    Args:
        feather_path: Path to the processed Feather file.
        start_dt: Start datetime for the filter (inclusive).
        end_dt: End datetime for the filter (exclusive).
        
    Returns:
        A new DataFrame containing only the filtered data.
    """
    logging.info(f"Loading data subset from {start_dt} to {end_dt}...")
    
    # Use DuckDB to query the Feather file directly on disk
    con = duckdb.connect()
    
    # The SQL query filters rows on the disk file based on the 'timestamp' column.
    sql_query = f"""
        SELECT *
        FROM read_parquet('{parquet_path}')
        WHERE 
            "timestamp" >= '{start_dt.isoformat()}' AND 
            "timestamp" < '{end_dt.isoformat()}'
    """
    
    # Fetch the filtered, smaller DataFrame
    df_subset = con.execute(sql_query).fetchdf()
    con.close()
    
    logging.info(f"✅ Loaded {len(df_subset):,} rows for the requested subset.")
    return df_subset

In [15]:
# Define the 30-day filter window
filter_start_date = datetime(2025, 9, 1)
filter_end_date = datetime(2025, 9, 2) # Exclusive end date

# 1. Load the filtered subset safely
df_subset = read_parquet_subset(
    parquet_path="../df_with_state.parquet",
    start_dt=filter_start_date,
    end_dt=filter_end_date
)

display(df_subset.head())

# 3. Aggressive memory cleanup after use (CRITICAL)
# del df_subset
# gc.collect()
# free_mem()
df_subset[df_subset.batt_mintemp>-40].batt_mintemp.describe()
df_subset.volt_delta_mv.describe(percentiles=[0.9,0.95,0.9992])
df_subset.timestamp.min(),df_subset.timestamp.max()

2025-11-26 18:18:23 - INFO - Loading data subset from 2025-09-01 00:00:00 to 2025-09-02 00:00:00...
2025-11-26 18:18:23 - INFO - ✅ Loaded 155,729 rows for the requested subset.


,id,reg_num,customer,model,timestamp,date_val,dt_sec,mode,vehiclereadycondition,gun_connection_status,batt_maxtemp,batt_mintemp,batt_temp_delta,maxtemp_bucket,temp_delta_bucket,batt_maxvolt,batt_minvolt,volt_delta_mv,volt_delta_bucket,batt_maxtemp_tc,batt_mintemp_tc,pack_id_max,pack_id_min,batt_maxvolt_cell,batt_minvolt_cell,bat_voltage,total_battery_current,bat_soc,soc_band_bucket,soh
0,18,DL1PD8509,ZB Gurgaon,12.5,2025-09-01 10:47:01.093000+05:30,2025-09-01 00:00:00+05:30,0.000,DISCHARGING,0,0,29,27,2,28–32,<2,3.287,3.276,11.0,10–20,8,18,1,2,136,503,629.900024,0.0,30.799999,30-40,99.599998
1,18,DL1PD8509,ZB Gurgaon,12.5,2025-09-01 10:47:03.232000+05:30,2025-09-01 00:00:00+05:30,2.139,DISCHARGING,0,0,29,27,2,28–32,<2,3.287,3.276,11.0,10–20,8,18,1,2,136,503,629.900024,0.0,30.799999,30-40,99.599998
2,18,DL1PD8509,ZB Gurgaon,12.5,2025-09-01 10:47:05.352000+05:30,2025-09-01 00:00:00+05:30,2.120,DISCHARGING,0,0,29,27,2,28–32,<2,3.288,3.276,12.0,10–20,8,18,1,2,518,277,629.900024,0.0,30.799999,30-40,99.599998
3,18,DL1PD8509,ZB Gurgaon,12.5,2025-09-01 10:47:07.592000+05:30,2025-09-01 00:00:00+05:30,2.240,DISCHARGING,0,0,29,27,2,28–32,<2,3.287,3.276,11.0,10–20,8,18,1,2,135,503,629.900024,0.0,30.799999,30-40,99.599998
4,18,DL1PD8509,ZB Gurgaon,12.5,2025-09-01 10:47:09.593000+05:30,2025-09-01 00:00:00+05:30,2.001,DISCHARGING,0,0,29,27,2,28–32,<2,3.287,3.276,11.0,10–20,8,18,1,2,135,277,629.900024,4.5,30.799999,30-40,99.599998


(Timestamp('2025-09-01 05:30:00.937000+0530', tz='Asia/Kolkata'),
 Timestamp('2025-09-01 23:59:59.826000+0530', tz='Asia/Kolkata'))

In [16]:
df_subset.volt_delta_mv.max()

np.float32(147.0)

In [17]:
df_subset.total_battery_current[df_subset.total_battery_current>-3200].min()

np.float32(-404.5)

In [18]:
# pd.set_option('display.float_format', '{:.2f}'.format)
# pd.reset_option('display.float_format')
df_subset.head(1000).to_csv('data_ref.csv')
df_subset[df_subset.dt_sec>1].dt_sec.describe(percentiles=[0.9,0.95,0.99,0.995,0.997]).round(2)

count    135658.00
mean          1.81
std           0.49
min           1.00
50%           2.06
90%           2.26
95%           2.32
99%           2.54
99.5%         2.72
99.7%         2.82
max           3.00
Name: dt_sec, dtype: float64

In [19]:
df_subset.total_battery_current.describe()

count    149022.000000
mean        -16.725883
std         144.110794
min        -404.500000
25%           0.000000
50%           3.200000
75%          19.299999
max         326.700012
Name: total_battery_current, dtype: float64

In [20]:
# df_subset[(df_subset.id=='3')&(df_subset.timestamp>'2025-09-01 16:12:48.737000+05:30')&(df_subset.timestamp<='2025-09-01 16:23:59.737000+05:30')]

In [21]:
df_subset.groupby(['id','date_val'])['dt_sec'].sum()/60.0

id  date_val                 
16  2025-09-01 00:00:00+05:30     959.589667
18  2025-09-01 00:00:00+05:30     477.738667
19  2025-09-01 00:00:00+05:30    1157.418783
3   2025-09-01 00:00:00+05:30    1756.423883
Name: dt_sec, dtype: float64

In [22]:
# # chk = ((df_subset.groupby(['date_val','mode','batt_maxtemp_tc','pack_id_max'])['dt_sec'].sum()*100.0/60.0)/(df_subset.groupby(['date_val'])['dt_sec'].sum()/60.0)).sort_values()
# chk = (
#         (df_subset[df_subset['mode'] == 'CHARGING'].groupby(['date_val','mode','batt_maxtemp_tc','pack_id_max'])['dt_sec'].sum()*100.0/60.0) / 
#         (df_subset[df_subset['mode'] == 'CHARGING'].groupby(['date_val'])['dt_sec'].sum()/60.0)
#       ).sort_index(level='batt_maxtemp_tc')
# display(chk)


# # chk = ((df_subset.groupby(['date_val','mode','batt_maxtemp_tc','pack_id_max'])['dt_sec'].sum()*100.0/60.0)/(df_subset.groupby(['date_val'])['dt_sec'].sum()/60.0)).sort_values()
# chk2 = (
#         (df_subset[df_subset['mode'] == 'CHARGING'].groupby(['date_val','mode','batt_maxtemp_tc','pack_id_max'])['dt_sec'].sum()/60.0)).sort_index(level='batt_maxtemp_tc')
# display(chk2)


In [23]:
# wa

In [24]:
# --- FIXED CATEGORY DEFINITIONS ---
# These fixed labels ensure your output columns are consistent across all data chunks.
FIXED_CATEGORIES = {
    'maxtemp_bucket': ["<28", "28–32", "32–35", "35–40", ">40"],
    'temp_delta_bucket': ["<2", "2–5", "5–8", ">8"],
    # 💥 UPDATED BUCKET: Using the new labels for volt_delta_bucket 💥
    'volt_delta_bucket': ["0–10", "10–20", "20–30", ">30"] 
}
# ----------------------------------

def create_session_report(
    df: pd.DataFrame, 
    timezone: str = 'Asia/Kolkata',
    min_discharging_duration_min: float = 1.0
) -> pd.DataFrame:
    """
    Performs sessionization and calculates the percentage distribution of 
    key operational buckets per session using a fixed set of categories.
    """
    # 1. CRITICAL STEP: Ensure data is sorted by ID and Timestamp
    df = df.sort_values(['id', 'timestamp']).reset_index(drop=True)
    
    # Ensure 'timestamp' is timezone-aware IST
    if not isinstance(df['timestamp'].dtype, pd.DatetimeTZDtype):
        try:
            df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce', utc=True).dt.tz_convert(timezone)
        except Exception:
            df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce')

    # 2. Identify the start of a new session
    df['id_change'] = df['id'].ne(df['id'].shift())
    df['mode_change'] = df['mode'].ne(df['mode'].shift())
    df['session_start'] = df['id_change'] | df['mode_change']
    df['unique_session_id'] = df['session_start'].cumsum()

    # 3. Aggregate Session Metrics (Non-bucket columns)
    session_df = df.groupby('unique_session_id').agg(
        id=('id', 'first'),
        reg_num=('reg_num', 'first'),
        customer=('customer', 'first'),
        model=('model', 'first'),
        mode=('mode', 'first'),
        start_time=('timestamp', 'min'),
        end_time=('timestamp', 'max'),
    ) # Unique_session_id is the index

    # 4. --- NEW: Calculate Percentage of Buckets per Session using FIXED CATEGORIES ---
    bucket_cols = ['maxtemp_bucket', 'temp_delta_bucket', 'volt_delta_bucket']
    agg_dfs = []
    
    for col in bucket_cols:
        if col in df.columns and col in FIXED_CATEGORIES:
            
            # --- Key Fix: Convert to CategoricalDtype with fixed categories ---
            # This ensures all categories are present, even those with zero count.
            df[col] = pd.Categorical(df[col], categories=FIXED_CATEGORIES[col], ordered=True)
            # -------------------------------------------------------------------
            
            # Calculate value counts normalized to get percentages
            pct_series = round(df.groupby('unique_session_id')[col].value_counts(normalize=True) * 100,2)
            
            # Unstack the bucket labels to create new columns, fill missing (zero) categories
            pivot_df = pct_series.unstack(fill_value=0)
            
            # Rename columns for clear identification and safe use (e.g., replaces '–' with '_')
            pivot_df.columns = [
                f"{col}_{str(c).replace('–', '_').replace('<', 'lt').replace('>', 'gt')}_pct" 
                for c in pivot_df.columns
            ]
            agg_dfs.append(pivot_df)

    # Merge all percentage tables back into the main session_df
    if agg_dfs:
        for agg_df in agg_dfs:
            session_df = session_df.join(agg_df, how='left')
    
    session_df = session_df.reset_index(drop=True) # Now reset index

    # 5. Final Calculations and Formatting
    
    # Calculate Duration in MINUTES (seconds / 60) and round to 2 decimal places
    session_df['duration'] = (
        (session_df['end_time'] - session_df['start_time']).dt.total_seconds() / 60
    ).round(2)

    # Add sequential session number (e.g., 1, 2, 3...) per vehicle ID
    session_df['session'] = session_df.groupby('id').cumcount() + 1
    
    # 6. Final Output Selection and Formatting
    
    session_report = session_df.copy()

    # Define the core columns to ensure they are first
    core_cols = ['id', 'reg_num', 'customer', 'model', 'mode', 'session', 'start_time', 'end_time', 'duration']
    
    # Order columns: core columns first, then new percentage columns
    new_cols = [c for c in session_report.columns if c not in core_cols]
    session_report = session_report[core_cols + new_cols]
    
    # --- FINAL FORMATTING ---
    # Round times to seconds precision (floor)
    session_report['start_time'] = session_report['start_time'].dt.floor('S')
    session_report['end_time'] = session_report['end_time'].dt.floor('S')
    session_report['date_val'] = session_report['start_time'].dt.date

    # Drop Timezone information (+05:30)
    session_report['start_time'] = session_report['start_time'].dt.tz_localize(None)
    session_report['end_time'] = session_report['end_time'].dt.tz_localize(None)
    # -------------------------

    return session_report

In [25]:
from datetime import datetime, timedelta
import pandas as pd
import gc
import logging # Required for managing log levels
from tqdm.auto import tqdm 
import time # For timing the execution

# NOTE: The create_session_report and read_parquet_subset functions 
# must be defined/imported before running this code.

def run_multi_day_session_analysis(
    parquet_path: str, 
    start_date: datetime, 
    num_days: int
) -> pd.DataFrame:
    """
    Iteratively loads, processes, and aggregates session reports for a defined number of days,
    displaying a progress bar and suppressing INFO logging during the process.
    """
    all_reports = []
    total_rows_ingested = 0 # <--- ADD THIS LINE HERE

    # --- Logging Suppression: Temporarily set log level to WARNING to hide INFO messages ---
    current_log_level = logging.getLogger().level
    logging.getLogger().setLevel(logging.WARNING) 
    
    try:
        progress_bar = tqdm(range(num_days), desc="Processing Daily Sessions", unit="day", colour="cyan") # Keeps the cyan color you had, distinct from the other function)
        
        for i in progress_bar:
            current_start_dt = start_date + timedelta(days=i)
            current_end_dt = current_start_dt + timedelta(days=1)
            
            progress_bar.set_postfix_str(f"Date: {current_start_dt.strftime('%Y-%m-%d')}")
            
            # 1. Load the small subset (Logging is suppressed here)
            df_subset = read_parquet_subset(
                parquet_path=parquet_path,
                start_dt=current_start_dt,
                end_dt=current_end_dt
            )
            
            if df_subset.empty:
                del df_subset
                gc.collect()
                continue
            
            total_rows_ingested += len(df_subset)   #count rows being processed

            # 2. Process the subset
            session_report_chunk = create_session_report(df_subset)
            
            # 3. Aggregate the result
            all_reports.append(session_report_chunk)
            
            # 4. Explicit Memory Management
            del df_subset 
            gc.collect()  
            
    finally:
        # --- Restore Logging Level ---
        logging.getLogger().setLevel(current_log_level)
        
    # 5. Final Concatenation
    if all_reports:
        final_report_df = pd.concat(all_reports, ignore_index=True)
        return final_report_df, total_rows_ingested
    else:
        return pd.DataFrame(), total_rows_ingested

# =========================================================================
# --- Execution Example ---
# =========================================================================

# Define the 75-day process window
filter_start_date = datetime(2025, 9, 1) # Start date
total_days_to_process = 75

print(f"--- Starting 75-Day Session Analysis ---")
print(f"Processing range: {filter_start_date.strftime('%Y-%m-%d')} to {(filter_start_date + timedelta(days=total_days_to_process-1)).strftime('%Y-%m-%d')}")
print("-" * 40)

start_time = time.time()

# Run the iterative analysis
session_report_df, total_rows = run_multi_day_session_analysis(
    parquet_path="../df_with_state.parquet",
    start_date=filter_start_date,
    num_days=total_days_to_process
)

end_time = time.time()
elapsed_time_sec = end_time - start_time
elapsed_time_min = elapsed_time_sec / 60

# --- Final Summary ---
print("\n" + "=" * 40)
print("✅ ANALYSIS COMPLETE")
print(f"Total time taken: {elapsed_time_sec:.2f} seconds ({elapsed_time_min:.2f} minutes)")
print(f"Days processed:   {total_days_to_process}")
print(f"Total Rows Ingested: {total_rows:,}")
print(f"Final Report Shape: {session_report_df.shape} (Rows: {session_report_df.shape[0]}, Columns: {session_report_df.shape[1]})")
print("=" * 40)

--- Starting 75-Day Session Analysis ---
Processing range: 2025-09-01 to 2025-11-14
----------------------------------------


Processing Daily Sessions: 100%|██████████| 75/75 [01:45<00:00,  1.40s/day, Date: 2025-11-14]


✅ ANALYSIS COMPLETE
Total time taken: 105.37 seconds (1.76 minutes)
Days processed:   75
Total Rows Ingested: 51,788,694
Final Report Shape: (18507, 23) (Rows: 18507, Columns: 23)


In [26]:
session_report_df.to_csv('bcs_analysis_sessions.csv')
session_report_df.head(5)

,id,reg_num,customer,model,mode,session,start_time,end_time,duration,maxtemp_bucket_lt28_pct,maxtemp_bucket_28_32_pct,maxtemp_bucket_32_35_pct,maxtemp_bucket_35_40_pct,maxtemp_bucket_gt40_pct,temp_delta_bucket_lt2_pct,temp_delta_bucket_2_5_pct,temp_delta_bucket_5_8_pct,temp_delta_bucket_gt8_pct,volt_delta_bucket_0_10_pct,volt_delta_bucket_10_20_pct,volt_delta_bucket_20_30_pct,volt_delta_bucket_gt30_pct,date_val
0,16,HR55AY9237,ZB Gurgaon,12.5,DISCHARGING,1,2025-09-01 05:30:00,2025-09-01 05:37:36,7.59,86.28,0.00,13.72,0.00,0.0,86.28,0.00,13.72,0.0,92.09,7.91,0.00,0.00,2025-09-01
1,16,HR55AY9237,ZB Gurgaon,12.5,CHARGING,2,2025-09-01 05:39:01,2025-09-01 06:13:04,34.05,0.00,0.00,38.89,61.11,0.0,0.00,0.05,99.95,0.0,7.15,92.80,0.05,0.00,2025-09-01
2,16,HR55AY9237,ZB Gurgaon,12.5,DISCHARGING,3,2025-09-01 06:13:05,2025-09-01 06:13:06,0.02,0.00,0.00,0.00,100.00,0.0,0.00,0.00,100.00,0.0,0.00,100.00,0.00,0.00,2025-09-01
3,16,HR55AY9237,ZB Gurgaon,12.5,CHARGING,4,2025-09-01 06:13:07,2025-09-01 07:22:50,69.71,0.00,16.74,32.25,51.02,0.0,0.00,0.00,100.00,0.0,0.00,49.76,21.03,29.21,2025-09-01
4,16,HR55AY9237,ZB Gurgaon,12.5,DISCHARGING,5,2025-09-01 07:23:05,2025-09-01 18:00:51,637.76,34.86,53.78,11.36,0.00,0.0,0.01,84.24,15.75,0.0,75.48,18.98,3.83,1.71,2025-09-01


In [27]:
import pandas as pd

def weighted_avg_factory(df, weight_col='duration'):
    """
    Creates a callable function for pandas aggregation that calculates 
    the weighted average of a series using a specific weight column from the 
    original DataFrame (df) based on the group's indices.
    """
    def weighted_avg(series):
        # Retrieve the original 'duration' (weights) for the rows in the current group (series)
        weights = df.loc[series.index, weight_col]
        
        # Guard against division by zero (e.g., if total duration is 0)
        if weights.sum() == 0:
            return 0.0
            
        # Weighted average calculation
        return ((series * weights).sum() / weights.sum()).round(4)

    return weighted_avg

# --- ASSUMING session_report_df is your input DataFrame (the 75-day report) ---
# NOTE: Replace 'session_report_df' with the actual variable name from your script (e.g., the output of run_multi_day_session_analysis)
df = session_report_df 

# Ensure 'duration' is numeric and set up the weighted average function
df['duration'] = pd.to_numeric(df['duration'], errors='coerce')
w_avg = weighted_avg_factory(df, 'duration')

# Define all percentage columns for aggregation
pct_cols = [col for col in df.columns if col.endswith('_pct')]

# Define the full aggregation dictionary
agg_dict = {
    'reg_num': 'first',
    'customer': 'first',
    'model': 'first',
    'session': 'count',  # Becomes total_sessions
    'duration': 'sum'   # Becomes total_duration
}

# Add weighted average for all percentage columns
weighted_agg_dict = {col: w_avg for col in pct_cols}
agg_dict.update(weighted_agg_dict)

In [28]:
# 1. Perform aggregation grouped by 'id' and 'mode'
report_agg = df.groupby(['id', 'mode', 'date_val']).agg(agg_dict).reset_index()

# 2. Final Formatting and Column Renaming/Reordering
report_agg = report_agg.rename(columns={'session': 'total_sessions', 'duration': 'total_duration'})
report_agg['total_duration'] = report_agg['total_duration'].round(2)

report_agg['maxtemp_bucket_lt28_pct'] = report_agg['maxtemp_bucket_lt28_pct'].round(2)
report_agg['maxtemp_bucket_28_32_pct'] = report_agg['maxtemp_bucket_28_32_pct'].round(2)
report_agg['maxtemp_bucket_32_35_pct'] = report_agg['maxtemp_bucket_32_35_pct'].round(2)
report_agg['maxtemp_bucket_35_40_pct'] = report_agg['maxtemp_bucket_35_40_pct'].round(2)
report_agg['maxtemp_bucket_gt40_pct'] = report_agg['maxtemp_bucket_gt40_pct'].round(2)

report_agg['temp_delta_bucket_lt2_pct'] = report_agg['temp_delta_bucket_lt2_pct'].round(2)
report_agg['temp_delta_bucket_2_5_pct'] = report_agg['temp_delta_bucket_2_5_pct'].round(2)
report_agg['temp_delta_bucket_5_8_pct'] = report_agg['temp_delta_bucket_5_8_pct'].round(2)
report_agg['temp_delta_bucket_gt8_pct'] = report_agg['temp_delta_bucket_gt8_pct'].round(2)

report_agg['volt_delta_bucket_0_10_pct'] = report_agg['volt_delta_bucket_0_10_pct'].round(2)
report_agg['volt_delta_bucket_10_20_pct'] = report_agg['volt_delta_bucket_10_20_pct'].round(2)
report_agg['volt_delta_bucket_20_30_pct'] = report_agg['volt_delta_bucket_20_30_pct'].round(2)
report_agg['volt_delta_bucket_gt30_pct'] = report_agg['volt_delta_bucket_gt30_pct'].round(2)


# Reorder columns as requested
final_columns = [
    'id', 'reg_num', 'customer', 'model', 'mode', 'date_val','total_sessions', 'total_duration'
] + pct_cols

report_agg = report_agg[final_columns]

In [29]:
report_agg.to_csv('bcs_analysis_report_v1.csv')
report_agg.head()

,id,reg_num,customer,model,mode,date_val,total_sessions,total_duration,maxtemp_bucket_lt28_pct,maxtemp_bucket_28_32_pct,maxtemp_bucket_32_35_pct,maxtemp_bucket_35_40_pct,maxtemp_bucket_gt40_pct,temp_delta_bucket_lt2_pct,temp_delta_bucket_2_5_pct,temp_delta_bucket_5_8_pct,temp_delta_bucket_gt8_pct,volt_delta_bucket_0_10_pct,volt_delta_bucket_10_20_pct,volt_delta_bucket_20_30_pct,volt_delta_bucket_gt30_pct
0,10,REGNUM_10,CUST_10,MDL_10,CHARGING,2025-09-30,3,219.73,42.55,57.45,0.00,0.0,0.0,46.04,53.96,0.00,0.0,48.06,32.38,6.42,13.14
1,10,REGNUM_10,CUST_10,MDL_10,CHARGING,2025-10-28,2,81.46,0.00,100.00,0.00,0.0,0.0,32.59,67.41,0.00,0.0,40.66,14.82,11.02,33.50
2,10,REGNUM_10,CUST_10,MDL_10,CHARGING,2025-10-29,8,130.99,13.30,86.70,0.00,0.0,0.0,0.00,78.88,21.12,0.0,100.00,0.00,0.00,0.00
3,10,REGNUM_10,CUST_10,MDL_10,CHARGING,2025-11-05,1,73.43,0.00,77.14,22.86,0.0,0.0,100.00,0.00,0.00,0.0,66.38,31.60,2.02,0.00
4,10,REGNUM_10,CUST_10,MDL_10,CHARGING,2025-11-06,4,116.11,36.43,40.49,23.08,0.0,0.0,61.43,38.57,0.00,0.0,80.02,17.84,2.14,0.00


In [30]:
report_agg[(report_agg.maxtemp_bucket_35_40_pct>40)|(report_agg.maxtemp_bucket_gt40_pct>40)].to_csv('bcs_session_maxT.csv')
report_agg[(report_agg.maxtemp_bucket_35_40_pct>40)|(report_agg.maxtemp_bucket_gt40_pct>40)].head()

,id,reg_num,customer,model,mode,date_val,total_sessions,total_duration,maxtemp_bucket_lt28_pct,maxtemp_bucket_28_32_pct,maxtemp_bucket_32_35_pct,maxtemp_bucket_35_40_pct,maxtemp_bucket_gt40_pct,temp_delta_bucket_lt2_pct,temp_delta_bucket_2_5_pct,temp_delta_bucket_5_8_pct,temp_delta_bucket_gt8_pct,volt_delta_bucket_0_10_pct,volt_delta_bucket_10_20_pct,volt_delta_bucket_20_30_pct,volt_delta_bucket_gt30_pct
20,11,AP39WF8593,FB Guntur,12.5,CHARGING,2025-09-05,1,53.10,0.0,0.00,14.33,85.67,0.00,0.00,100.00,0.00,0.0,20.24,79.02,0.74,0.00
21,11,AP39WF8593,FB Guntur,12.5,CHARGING,2025-09-06,5,332.99,0.0,0.71,34.48,56.00,8.82,10.49,89.51,0.00,0.0,31.82,31.04,7.50,29.64
22,11,AP39WF8593,FB Guntur,12.5,CHARGING,2025-09-07,6,264.72,0.0,7.14,39.21,43.42,10.24,8.01,81.49,10.49,0.0,44.57,42.38,2.93,10.11
23,11,AP39WF8593,FB Guntur,12.5,CHARGING,2025-09-08,117,277.15,0.0,2.89,27.85,65.32,3.94,4.47,68.09,27.44,0.0,49.87,30.55,2.63,16.94
34,11,AP39WF8593,FB Guntur,12.5,CHARGING,2025-09-19,4,244.54,0.0,3.06,20.42,65.94,10.57,4.84,78.58,16.59,0.0,37.68,31.34,2.33,28.64


In [31]:
report_agg[(report_agg.temp_delta_bucket_5_8_pct>40)|(report_agg.temp_delta_bucket_gt8_pct>40)].to_csv('bcs_session_deltaT.csv')
report_agg[(report_agg.temp_delta_bucket_5_8_pct>40)|(report_agg.temp_delta_bucket_gt8_pct>40)].head()

,id,reg_num,customer,model,mode,date_val,total_sessions,total_duration,maxtemp_bucket_lt28_pct,maxtemp_bucket_28_32_pct,maxtemp_bucket_32_35_pct,maxtemp_bucket_35_40_pct,maxtemp_bucket_gt40_pct,temp_delta_bucket_lt2_pct,temp_delta_bucket_2_5_pct,temp_delta_bucket_5_8_pct,temp_delta_bucket_gt8_pct,volt_delta_bucket_0_10_pct,volt_delta_bucket_10_20_pct,volt_delta_bucket_20_30_pct,volt_delta_bucket_gt30_pct
49,11,AP39WF8593,FB Guntur,12.5,CHARGING,2025-10-04,4,201.55,0.00,6.41,43.60,49.99,0.00,2.29,50.29,47.42,0.0,32.91,54.73,5.20,7.16
51,11,AP39WF8593,FB Guntur,12.5,CHARGING,2025-10-06,7,278.92,0.01,22.80,21.52,44.23,11.43,6.38,41.26,52.36,0.0,53.35,38.71,2.08,5.86
58,11,AP39WF8593,FB Guntur,12.5,CHARGING,2025-10-13,7,283.56,0.00,23.61,25.78,50.61,0.00,0.00,44.89,55.11,0.0,34.30,36.46,3.21,26.03
60,11,AP39WF8593,FB Guntur,12.5,CHARGING,2025-10-15,3,147.83,0.00,0.01,14.18,85.81,0.00,0.02,27.85,72.13,0.0,43.41,41.69,3.86,11.04
62,11,AP39WF8593,FB Guntur,12.5,CHARGING,2025-10-17,4,199.44,0.63,10.01,37.42,51.94,0.00,0.08,54.95,44.97,0.0,35.26,53.20,2.80,8.74


In [32]:
report_agg[(report_agg.volt_delta_bucket_20_30_pct>40)|(report_agg.volt_delta_bucket_gt30_pct>40)].to_csv('bcs_session_deltaV.csv')
report_agg[(report_agg.volt_delta_bucket_20_30_pct>40)|(report_agg.volt_delta_bucket_gt30_pct>40)].head()

,id,reg_num,customer,model,mode,date_val,total_sessions,total_duration,maxtemp_bucket_lt28_pct,maxtemp_bucket_28_32_pct,maxtemp_bucket_32_35_pct,maxtemp_bucket_35_40_pct,maxtemp_bucket_gt40_pct,temp_delta_bucket_lt2_pct,temp_delta_bucket_2_5_pct,temp_delta_bucket_5_8_pct,temp_delta_bucket_gt8_pct,volt_delta_bucket_0_10_pct,volt_delta_bucket_10_20_pct,volt_delta_bucket_20_30_pct,volt_delta_bucket_gt30_pct
25,11,AP39WF8593,FB Guntur,12.5,CHARGING,2025-09-10,2,757.11,0.00,78.02,21.98,0.00,0.0,73.86,25.97,0.16,0.0,20.20,5.51,0.60,73.69
26,11,AP39WF8593,FB Guntur,12.5,CHARGING,2025-09-11,4,305.07,0.01,56.51,43.49,0.00,0.0,27.12,72.88,0.00,0.0,21.61,16.01,1.35,61.03
32,11,AP39WF8593,FB Guntur,12.5,CHARGING,2025-09-17,2,787.06,0.00,63.60,36.40,0.00,0.0,50.36,49.64,0.00,0.0,35.61,9.37,0.98,54.04
39,11,AP39WF8593,FB Guntur,12.5,CHARGING,2025-09-24,1,120.25,0.00,0.00,68.69,31.31,0.0,0.00,83.23,16.77,0.0,9.94,16.95,1.31,71.80
44,11,AP39WF8593,FB Guntur,12.5,CHARGING,2025-09-29,5,264.25,0.30,4.14,48.33,47.22,0.0,7.83,71.65,20.52,0.0,27.82,18.25,1.07,52.87


In [33]:
import pandas as pd
import numpy as np

def compute_detailed_hotspot(df, item_id_col='tc_id', pack_id_col='pack_id'):
    """
    Computes detailed hotspot analysis for a single vehicle on a single day.
    
    Args:
        df (pd.DataFrame): Input DataFrame for a single vehicle and day.
        item_id_col (str): The column representing the individual item ID 
                           (e.g., 'tc_id' or 'pack_id').
        pack_id_col (str): The column representing the pack ID (e.g., 'pack_id').
                           This is separated for TC-level grouping.
    
    Returns:
        pd.DataFrame: Detailed hotspot metrics grouped by item_id and pack_id.
    """
    if df.empty:
        return pd.DataFrame()

    # Basic Info
    date_val = df['date_val'].iloc[0]
    vid = df['id'].iloc[0]
    
    # Pre-calculate total time for the day (in seconds)
    total_day_sec = df['dt_sec'].sum()
    # total_day_mins = min(total_day_sec / 60.0, 1440.0) # Clipping the daily duration
    total_day_mins = total_day_sec / 60.0
    
    # --- Helper to aggregate ---
    def get_agg(subset_df, id_col, val_col, pack_col):
        if subset_df.empty:
            return pd.Series(dtype='float64')
        # Group by the item ID and its associated Pack ID
        return subset_df.groupby([id_col, pack_col])[val_col].sum()

    # Determine the columns to group by (Max TC/Pack and Min TC/Pack)
    # The columns in the raw data are fixed: batt_maxtemp_tc, pack_id_max, etc.
    group_cols = [item_id_col, pack_id_col]

    # Use the relevant columns from the raw data for aggregation:
    c_df = df[df['mode'] == 'CHARGING']
    dc_df = df[df['mode'] == 'DISCHARGING']
    
    # 1. C-Max (Charging, Max Temp)
    s_cmax = get_agg(c_df, 'batt_maxtemp_tc', 'dt_sec', 'pack_id_max')
    
    # 2. DC-Max (Discharging, Max Temp)
    s_dcmax = get_agg(dc_df, 'batt_maxtemp_tc', 'dt_sec', 'pack_id_max')
    
    # 3. C-Min (Charging, Min Temp)
    s_cmin = get_agg(c_df, 'batt_mintemp_tc', 'dt_sec', 'pack_id_min')
    
    # 4. DC-Min (Discharging, Min Temp)
    s_dcmin = get_agg(dc_df, 'batt_mintemp_tc', 'dt_sec', 'pack_id_min')
    
    # --- Merge all series ---
    dfs = []
    
    # Combine the index names for consistent merging (TC ID and Pack ID)
    index_names = ['tc_id', 'pack_id'] 

    if not s_cmax.empty:
        s_cmax.index.names = index_names
        dfs.append(s_cmax.rename('cmax_dur'))
        
    if not s_dcmax.empty:
        s_dcmax.index.names = index_names
        dfs.append(s_dcmax.rename('dcmax_dur'))
        
    if not s_cmin.empty:
        s_cmin.index.names = index_names
        dfs.append(s_cmin.rename('cmin_dur'))
        
    if not s_dcmin.empty:
        s_dcmin.index.names = index_names
        dfs.append(s_dcmin.rename('dcmin_dur'))
        
    if not dfs:
        return pd.DataFrame()
        
    merged = pd.concat(dfs, axis=1).fillna(0)
    
    # Ensure all duration columns exist
    target_dur_cols = ['cmax_dur', 'dcmax_dur', 'cmin_dur', 'dcmin_dur']
    merged = merged.reindex(columns=list(set(merged.columns) | set(target_dur_cols)), fill_value=0)
    
    # --- Calculations ---
    
    # Assign the clipped total time to the daily_dur column
    merged['daily_dur'] = total_day_mins 

    # Convert seconds to minutes for Duration columns
    merged[['cmax_dur', 'dcmax_dur', 'cmin_dur', 'dcmin_dur']] = merged[['cmax_dur', 'dcmax_dur', 'cmin_dur', 'dcmin_dur']] / 60.0
    
    # Calculate Percentages against the Total Day Time (mode-agnostic)
    if total_day_mins > 0:
        for col in target_dur_cols:
            merged[f'{col}_pct'] = (merged[col] / total_day_mins) * 100.0
    else:
        for col in target_dur_cols:
            merged[f'{col}_pct'] = 0.0
        
    # --- Formatting ---
    cols_to_round = ['daily_dur','cmax_dur', 'cmax_dur_pct', 'dcmax_dur', 'dcmax_dur_pct', 
                     'cmin_dur', 'cmin_dur_pct', 'dcmin_dur', 'dcmin_dur_pct']
    
    # Ensure all percentage columns exist
    for col in cols_to_round:
        if col not in merged.columns:
             merged[col] = 0.0
             
    merged[cols_to_round] = merged[cols_to_round].round(2)
    
    # Reset index and insert identifier columns
    merged = merged.reset_index()
    merged.insert(0, 'id', vid)
    merged.insert(0, 'date_val', date_val)
    
    # Reorder columns
    desired_order = ['date_val', 'id', 'tc_id', 'pack_id', 'daily_dur',
                     'cmax_dur', 'cmax_dur_pct', 'dcmax_dur', 'dcmax_dur_pct',
                     'cmin_dur', 'cmin_dur_pct', 'dcmin_dur', 'dcmin_dur_pct']
    
    return merged[desired_order]

In [34]:
# df_sub = read_parquet_subset("df_with_state.parquet", datetime(2025, 9, 1), datetime(2025, 9, 2))
# df_sub.head(500).to_csv('hotspot_ip.csv')

In [35]:
def run_multi_day_tc_hotspot_analysis(
    parquet_path: str, 
    start_date: datetime, 
    num_days: int
):
    all_tables = []
    total_rows = 0

    old_level = logging.getLogger().level
    logging.getLogger().setLevel(logging.WARNING)

    try: 
        for d in tqdm(range(num_days), desc="Processing Days", ncols=70):

            day_start = start_date + timedelta(days=d)
            day_end   = day_start + timedelta(days=1)

            df_sub = read_parquet_subset(parquet_path, day_start, day_end)
            if df_sub.empty:
                continue

            total_rows += len(df_sub)
            df_sub["date_val"] = day_start.date()

            # per-vehicle
            for vid, g in df_sub.groupby("id"):
                res = compute_detailed_hotspot(g)
                if not res.empty:
                    res["id"] = vid
                    all_tables.append(res)

            del df_sub
            gc.collect()

    finally:
        logging.getLogger().setLevel(old_level)

    if not all_tables:
        return pd.DataFrame(), total_rows

    return pd.concat(all_tables, ignore_index=True), total_rows


In [36]:
start_date = datetime(2025, 9, 1)

tc_hotspot_df, total_rows = run_multi_day_tc_hotspot_analysis(
    parquet_path="../df_with_state.parquet",
    start_date=start_date,
    num_days=75
)

print(tc_hotspot_df.head())
print("Total rows processed:", total_rows)


Processing Days: 100%|████████████████| 75/75 [01:35<00:00,  1.27s/it]

     date_val  id  tc_id  pack_id  daily_dur  cmax_dur  cmax_dur_pct  \
0  2025-09-01  16      5        1     959.59      2.81          0.29   
1  2025-09-01  16      8        1     959.59      0.04          0.00   
2  2025-09-01  16     17        2     959.59      2.74          0.29   
3  2025-09-01  16     73        9     959.59      0.05          0.01   
4  2025-09-01  16     80        9     959.59    228.08         23.77   

   dcmax_dur  dcmax_dur_pct  cmin_dur  cmin_dur_pct  dcmin_dur  dcmin_dur_pct  
0       0.04           0.00       0.0           0.0      27.99           2.92  
1       0.04           0.00       0.0           0.0      58.46           6.09  
2       0.56           0.06       0.0           0.0       0.00           0.00  
3      52.38           5.46       0.0           0.0       0.00           0.00  
4     270.21          28.16       0.0           0.0       0.00           0.00  
Total rows processed: 51788694


In [37]:
sort_cols = ['date_val', 'id', 'pack_id', 'tc_id']
tc_hotspot_df = tc_hotspot_df.sort_values(by=sort_cols, ascending=True).reset_index(drop=True)
tc_hotspot_df.to_csv('hotspot.csv')
display(tc_hotspot_df.head())

,date_val,id,tc_id,pack_id,daily_dur,cmax_dur,cmax_dur_pct,dcmax_dur,dcmax_dur_pct,cmin_dur,cmin_dur_pct,dcmin_dur,dcmin_dur_pct
0,2025-09-01,16,3,1,959.59,0.00,0.00,6.57,0.68,0.00,0.00,0.00,0.00
1,2025-09-01,16,5,1,959.59,2.81,0.29,0.04,0.00,0.00,0.00,27.99,2.92
2,2025-09-01,16,7,1,959.59,0.00,0.00,0.00,0.00,29.69,3.09,71.96,7.50
3,2025-09-01,16,8,1,959.59,0.04,0.00,0.04,0.00,0.00,0.00,58.46,6.09
4,2025-09-01,16,9,1,959.59,0.00,0.00,0.00,0.00,189.47,19.74,510.59,53.21


In [38]:
pd.set_option('display.max_rows', 10)
tc_hotspot_df[tc_hotspot_df.daily_dur>1440].daily_dur.unique()

array([1756.42, 1460.31, 2382.81, 1918.67])

In [39]:
# pd.set_option('display.max_rows', 100)
# tc_hotspot_df.groupby(['date_val','id','daily_dur','pack_id'])[['cmax_dur_pct','dcmax_dur_pct','cmin_dur_pct','dcmin_dur_pct']].sum().round(2).head(7)

In [40]:
pd.set_option('display.max_rows', None)
import pandas as pd

# Load the TC-level aggregated data (hotspot.csv)
df_hotspot = pd.read_csv('hotspot.csv')

# 1. Group by pack_id and sum all duration columns (NUMERATOR)
pack_agg = df_hotspot.groupby(['pack_id']).agg(
    total_cmax_dur=('cmax_dur', 'sum'),
    total_dcmax_dur=('dcmax_dur', 'sum'),
    total_cmin_dur=('cmin_dur', 'sum'),
    total_dcmin_dur=('dcmin_dur', 'sum'),
).reset_index()

# 2. Calculate the Grand Total for each duration column (DENOMINATOR)
grand_totals = pack_agg[['total_cmax_dur', 'total_dcmax_dur', 
                         'total_cmin_dur', 'total_dcmin_dur']].sum()

# 3. Calculate the percentage share for each pack
for col in ['cmax_dur', 'dcmax_dur', 'cmin_dur', 'dcmin_dur']:
    # Get the total duration for this specific metric
    total_duration = grand_totals[f'total_{col}']
    
    if total_duration > 0:
        # Calculate the pack's share of the total problem duration
        pack_agg[f'{col}_Share_Pct'] = (
            pack_agg[f'total_{col}'] / total_duration * 100
        ).round(2)
    else:
        pack_agg[f'{col}_Share_Pct'] = 0.0

# 4. Select the final output columns
hotspot_pack_level = pack_agg[['pack_id', 
                             'cmax_dur_Share_Pct', 'dcmax_dur_Share_Pct',
                             'cmin_dur_Share_Pct', 'dcmin_dur_Share_Pct']]

hotspot_pack_level.to_csv('hotspot_pack.csv')

In [41]:
import pandas as pd
import numpy as np

def generate_high_delta_v_report(df: pd.DataFrame) -> pd.DataFrame:
    """
    Analyzes charging sessions for high DeltaV (> 20mV) and aggregates metrics 
    by SOC band across the entire fleet.
    """
    
    # Ensure necessary columns are correctly typed
    df['volt_delta_mv'] = pd.to_numeric(df['volt_delta_mv'], errors='coerce')
    df['total_battery_current'] = pd.to_numeric(df['total_battery_current'], errors='coerce')
    if 'dt_sec' in df.columns:
        df['dt_sec'] = pd.to_numeric(df['dt_sec'], errors='coerce')

    # --- 1. Filter Data ---
    df_charging = df[df['mode'] == 'CHARGING'].copy()
    df_filtered = df_charging[df_charging['volt_delta_mv'] > 20].copy()
    
    # Prepare absolute current column
    df_filtered['abs_current'] = df_filtered['total_battery_current'].abs()

    # --- FIX 1: Update the column name in the final structure list ---
    final_report_cols_structure = [
        'total_ids', 'SOC band', '% Time', 
        'mv Imbalance (median)', 'mv Imbalance (P95)', # <--- FIXED: Now uses P95
        'Charging Current (Median)', 'Charging Current (Max)'
    ]
    # -----------------------------------------------------------------

    if df_filtered.empty:
        # Return an empty DataFrame with the correct structure
        return pd.DataFrame(columns=final_report_cols_structure)

    # --- 2. Calculate Total Vehicle Count ---
    total_unique_vehicles = df_filtered['id'].nunique()

    p95_func = lambda x: x.quantile(0.95)    

    # --- 3. Grouping and Aggregation ---
    grouped_by_soc = df_filtered.groupby('soc_band_bucket').agg(
        **{
            'mv Imbalance (median)': ('volt_delta_mv', 'median'),
            'mv Imbalance (P95)': ('volt_delta_mv', p95_func), # P95 is calculated here
            'Charging Current (Median)': ('abs_current', 'median'),
            'Charging Current (Max)': ('abs_current', 'max'),
            'Band Duration (sec)': ('dt_sec', 'sum')            
        }
    ).reset_index()

    # --- CALCULATE % TIME ---
    total_time_filtered = grouped_by_soc['Band Duration (sec)'].sum()

    if total_time_filtered > 0:
        grouped_by_soc['% Time'] = (
            grouped_by_soc['Band Duration (sec)'] / total_time_filtered * 100
        ).round(2)
    else:
        grouped_by_soc['% Time'] = 0.0
        
    # Drop the temporary duration column
    grouped_by_soc = grouped_by_soc.drop(columns=['Band Duration (sec)'])

    # --- 4. Final Formatting ---
    final_report = grouped_by_soc.rename(columns={'soc_band_bucket': 'SOC band'})
    
    # Insert the 'Vehicle ID' column and assign the total count
    final_report.insert(0, 'total_ids', total_unique_vehicles)

    # --- FIX 2: Update rounding logic to include P95 ---
    for col in final_report.columns:
        # Check for 'median', 'P95', 'Max', or 'Time' to round all metrics
        if 'median' in col or 'P95' in col or 'Max' in col or 'Time' in col:
            final_report[col] = final_report[col].round(2)
    # ----------------------------------------------------
            
    # Ensure final column order is correct (now that P95 is in the structure list)
    final_report = final_report[[col for col in final_report_cols_structure if col in final_report.columns]]

    return final_report

In [42]:
def run_multi_day_deltav_analysis(
    parquet_path: str, 
    start_date: datetime, 
    num_days: int
):
    """
    Reads multi-day Parquet data and computes the fleet-wide high DeltaV report, 
    using a progress bar for the outer loop.
    """
    raw_data_for_deltav = []
    total_rows = 0

    # Logging setup (remains the same to avoid NameError)
    old_level = logging.getLogger().level
    logging.getLogger().setLevel(logging.WARNING)

    try: 
        # --- CHANGE HERE: Wrap range(num_days) with tqdm ---
        # NOTE: You need to have 'from tqdm import tqdm' imported in your environment
        # Use a descriptive label like "DeltaV Analysis Days"
        for d in tqdm(range(num_days), desc="DeltaV Analysis Days", ncols=70): 
        # --------------------------------------------------

            day_start = start_date + timedelta(days=d)
            day_end   = day_start + timedelta(days=1)

            # --- Replace with your actual Parquet reading function ---
            df_sub = read_parquet_subset(parquet_path, day_start, day_end)
            if df_sub.empty:
                continue

            total_rows += len(df_sub)
            
            # --- COLLECT RAW DATA ---
            cols_needed = ['id', 'dt_sec','mode', 'volt_delta_mv', 'total_battery_current', 'soc_band_bucket']
            cols_present = [col for col in cols_needed if col in df_sub.columns]
            
            raw_data_for_deltav.append(df_sub[cols_present].copy())

            del df_sub
            gc.collect()

    finally:
        logging.getLogger().setLevel(old_level)

    # -------------------------------------------------------------
    # RUN FLEET-WIDE DELTAV ANALYSIS (ONCE)
    # -------------------------------------------------------------
    if raw_data_for_deltav:
        full_raw_data_df = pd.concat(raw_data_for_deltav, ignore_index=True)
        delta_v_report_df = generate_high_delta_v_report(full_raw_data_df)
    else:
        delta_v_report_df = generate_high_delta_v_report(pd.DataFrame())

    return delta_v_report_df, total_rows


final_report, total_rows = run_multi_day_deltav_analysis(
    parquet_path="../df_with_state.parquet",
    start_date=datetime(2025, 9, 1),
    num_days=75
)

DeltaV Analysis Days: 100%|███████████| 75/75 [01:10<00:00,  1.06it/s]


In [43]:
final_report

,total_ids,SOC band,% Time,mv Imbalance (median),mv Imbalance (P95),Charging Current (Median),Charging Current (Max)
0,28,10–20,0.09,23.0,26.0,370.399994,382.899994
1,28,20–30,0.15,22.0,28.0,367.299988,405.799988
2,28,30-40,0.18,22.0,28.0,366.799988,405.500000
3,28,40-50,0.37,22.0,24.0,365.600006,404.200012
4,28,50-60,1.08,22.0,29.0,362.299988,502.899994
5,28,60-70,1.06,22.0,29.0,316.600006,408.200012
6,28,70-80,1.03,22.0,37.0,358.100006,492.600006
7,28,80-90,14.65,26.0,80.0,331.799988,503.000000
8,28,90-100,81.40,60.0,122.0,91.500000,493.899994


In [44]:
final_report.to_csv('soc_band_mv_current_analysis.csv')

In [45]:
import pandas as pd
import numpy as np

def generate_thermal_voltage_hotspot_report(df: pd.DataFrame) -> pd.DataFrame:
    """
    Generates a report showing Thermocouple (TC) Hotspot exposure duration 
    (Max Temp/Min Temp) during high voltage imbalance (volt_delta_mv > 5).
    
    Required columns: dt_sec, volt_delta_mv, batt_maxtemp_tc, batt_mintemp_tc.
    """
    
    # --- 1. Data Type Conversion and Preparation ---
    df['volt_delta_mv'] = pd.to_numeric(df['volt_delta_mv'], errors='coerce')
    
    # Check for duration column and convert to minutes
    if 'dt_sec' not in df.columns:
        print("Error: 'dt_sec' column missing for duration calculation.")
        return pd.DataFrame() 

    df['dt_min'] = pd.to_numeric(df['dt_sec'], errors='coerce') / 60.0

    # --- 2. Filtering ---
    # Apply condition: volt_delta_mv > 5 (High Voltage Imbalance)
    df_filtered = df[df['volt_delta_mv'] > 5].copy()
    
    final_report_cols_structure = [
        'Thermocouple ID', 'Max Temp Duration (min)', 'Min Temp Duration (min)'
    ]

    if df_filtered.empty:
        return pd.DataFrame(columns=final_report_cols_structure)

    # --- 3. Aggregation for MAX Temperature Duration ---
    # Group by the TC ID that was reporting the MAX temp and sum the duration.
    max_temp_df = df_filtered.groupby('batt_maxtemp_tc')['dt_min'].sum().reset_index()
    max_temp_df = max_temp_df.rename(columns={
        'batt_maxtemp_tc': 'Thermocouple ID',
        'dt_min': 'Max Temp Duration (min)'
    })

    # --- 4. Aggregation for MIN Temperature Duration ---
    # Group by the TC ID that was reporting the MIN temp and sum the duration.
    min_temp_df = df_filtered.groupby('batt_mintemp_tc')['dt_min'].sum().reset_index()
    min_temp_df = min_temp_df.rename(columns={
        'batt_mintemp_tc': 'Thermocouple ID',
        'dt_min': 'Min Temp Duration (min)'
    })

    # --- 5. Merge and Cleanup ---
    # Perform an Outer Join to combine both results, keeping all TCs
    final_report = pd.merge(
        max_temp_df, 
        min_temp_df, 
        on='Thermocouple ID', 
        how='outer'
    )

    # Fill NaNs created by the outer merge with 0
    final_report = final_report.fillna(0)

    # Round all duration metrics
    for col in final_report.columns:
        if 'Duration' in col:
            final_report[col] = final_report[col].round(2)
            
    # Filter out TCs with zero total exposure
    final_report = final_report[
        (final_report['Max Temp Duration (min)'] > 0) | 
        (final_report['Min Temp Duration (min)'] > 0)
    ]
    
    # Ensure ID column is integer (since TC IDs are usually integers)
    final_report['Thermocouple ID'] = final_report['Thermocouple ID'].astype(pd.Int64Dtype())

    return final_report


def run_multi_day_deltav_analysis(
    parquet_path: str, 
    start_date: datetime, 
    num_days: int
):
    """
    Reads multi-day Parquet data and computes the fleet-wide high DeltaV report, 
    using a progress bar for the outer loop.
    """
    raw_data = []
    total_rows = 0

    # Logging setup (remains the same to avoid NameError)
    old_level = logging.getLogger().level
    logging.getLogger().setLevel(logging.WARNING)

    try: 
        # --- CHANGE HERE: Wrap range(num_days) with tqdm ---
        # NOTE: You need to have 'from tqdm import tqdm' imported in your environment
        # Use a descriptive label like "DeltaV Analysis Days"
        for d in tqdm(range(num_days), desc="ThermalV Analysis Days", ncols=70): 
        # --------------------------------------------------

            day_start = start_date + timedelta(days=d)
            day_end   = day_start + timedelta(days=1)

            # --- Replace with your actual Parquet reading function ---
            df_sub = read_parquet_subset(parquet_path, day_start, day_end)
            if df_sub.empty:
                continue

            total_rows += len(df_sub)
            
            # --- COLLECT RAW DATA ---
            cols_needed = ['batt_maxtemp_tc','batt_mintemp_tc', 'dt_sec', 'volt_delta_mv']
            cols_present = [col for col in cols_needed if col in df_sub.columns]
            
            raw_data.append(df_sub[cols_present].copy())

            del df_sub
            gc.collect()

    finally:
        logging.getLogger().setLevel(old_level)

    # -------------------------------------------------------------
    # RUN FLEET-WIDE DELTAV ANALYSIS (ONCE)
    # -------------------------------------------------------------
    if raw_data:
        full_raw_data_df = pd.concat(raw_data, ignore_index=True)
        delta_v_report_df = generate_thermal_voltage_hotspot_report(full_raw_data_df)
    else:
        delta_v_report_df = generate_thermal_voltage_hotspot_report(pd.DataFrame())

    return delta_v_report_df, total_rows


thermal_v_tc_hotspot_report, total_rows = run_multi_day_deltav_analysis(
    parquet_path="../df_with_state.parquet",
    start_date=datetime(2025, 9, 1),
    num_days=75
)

display(thermal_v_tc_hotspot_report.head())


ThermalV Analysis Days: 100%|█████████| 75/75 [01:12<00:00,  1.04it/s]


,Thermocouple ID,Max Temp Duration (min),Min Temp Duration (min)
0,1,10.71,4346.16
1,2,601.45,4320.89
2,3,554.84,131.61
3,4,1456.65,26.34
4,5,5227.75,7605.25


In [46]:
thermal_v_tc_hotspot_report.to_csv('hotspot_tc.csv')

In [47]:
import pandas as pd
import numpy as np

def generate_thermal_voltage_pack_hotspot_report(df: pd.DataFrame) -> pd.DataFrame:
    """
    Generates a report showing Pack Hotspot exposure duration 
    (Max Temp/Min Temp) during high voltage imbalance (volt_delta_mv > 5).
    
    Required columns: dt_sec, volt_delta_mv, batt_maxtemp_tc, batt_mintemp_tc.
    """
    
    # --- 1. Data Type Conversion and Preparation ---
    df['volt_delta_mv'] = pd.to_numeric(df['volt_delta_mv'], errors='coerce')
    
    # Check for duration column and convert to minutes
    if 'dt_sec' not in df.columns:
        print("Error: 'dt_sec' column missing for duration calculation.")
        return pd.DataFrame() 

    df['dt_min'] = pd.to_numeric(df['dt_sec'], errors='coerce') / 60.0

    # --- 2. Filtering ---
    # Apply condition: volt_delta_mv > 5 (High Voltage Imbalance)
    df_filtered = df[df['volt_delta_mv'] > 5].copy()
    
    final_report_cols_structure = [
        'Pack ID', 'Max Temp Duration (min)', 'Min Temp Duration (min)'
    ]

    if df_filtered.empty:
        return pd.DataFrame(columns=final_report_cols_structure)

    # --- 3. Aggregation for MAX Temperature Duration ---
    # Group by the TC ID that was reporting the MAX temp and sum the duration.
    max_temp_df = df_filtered.groupby('pack_id_max')['dt_min'].sum().reset_index()
    max_temp_df = max_temp_df.rename(columns={
        'pack_id_max': 'Pack ID',
        'dt_min': 'Max Temp Duration (min)'
    })

    # --- 4. Aggregation for MIN Temperature Duration ---
    # Group by the TC ID that was reporting the MIN temp and sum the duration.
    min_temp_df = df_filtered.groupby('pack_id_min')['dt_min'].sum().reset_index()
    min_temp_df = min_temp_df.rename(columns={
        'pack_id_min': 'Pack ID',
        'dt_min': 'Min Temp Duration (min)'
    })

    # --- 5. Merge and Cleanup ---
    # Perform an Outer Join to combine both results, keeping all TCs
    final_report = pd.merge(
        max_temp_df, 
        min_temp_df, 
        on='Pack ID', 
        how='outer'
    )

    # Fill NaNs created by the outer merge with 0
    final_report = final_report.fillna(0)

    # Round all duration metrics
    for col in final_report.columns:
        if 'Duration' in col:
            final_report[col] = final_report[col].round(2)
            
    # Filter out TCs with zero total exposure
    final_report = final_report[
        (final_report['Max Temp Duration (min)'] > 0) | 
        (final_report['Min Temp Duration (min)'] > 0)
    ]
    
    # Ensure ID column is integer (since TC IDs are usually integers)
    final_report['Pack ID'] = final_report['Pack ID'].astype(pd.Int64Dtype())

    return final_report


def run_multi_day_deltav_analysis(
    parquet_path: str, 
    start_date: datetime, 
    num_days: int
):
    """
    Reads multi-day Parquet data and computes the fleet-wide high DeltaV report, 
    using a progress bar for the outer loop.
    """
    raw_data = []
    total_rows = 0

    # Logging setup (remains the same to avoid NameError)
    old_level = logging.getLogger().level
    logging.getLogger().setLevel(logging.WARNING)

    try: 
        # --- CHANGE HERE: Wrap range(num_days) with tqdm ---
        # NOTE: You need to have 'from tqdm import tqdm' imported in your environment
        # Use a descriptive label like "DeltaV Analysis Days"
        for d in tqdm(range(num_days), desc="ThermalV Analysis Days", ncols=70): 
        # --------------------------------------------------

            day_start = start_date + timedelta(days=d)
            day_end   = day_start + timedelta(days=1)

            # --- Replace with your actual Parquet reading function ---
            df_sub = read_parquet_subset(parquet_path, day_start, day_end)
            if df_sub.empty:
                continue

            total_rows += len(df_sub)
            
            # --- COLLECT RAW DATA ---
            cols_needed = ['pack_id_max','pack_id_min', 'dt_sec', 'volt_delta_mv']
            cols_present = [col for col in cols_needed if col in df_sub.columns]
            
            raw_data.append(df_sub[cols_present].copy())

            del df_sub
            gc.collect()

    finally:
        logging.getLogger().setLevel(old_level)

    # -------------------------------------------------------------
    # RUN FLEET-WIDE DELTAV ANALYSIS (ONCE)
    # -------------------------------------------------------------
    if raw_data:
        full_raw_data_df = pd.concat(raw_data, ignore_index=True)
        delta_v_report_df = generate_thermal_voltage_pack_hotspot_report(full_raw_data_df)
    else:
        delta_v_report_df = generate_thermal_voltage_pack_hotspot_report(pd.DataFrame())

    return delta_v_report_df, total_rows


thermal_v_pack_hotspot_report, total_rows = run_multi_day_deltav_analysis(
    parquet_path="../df_with_state.parquet",
    start_date=datetime(2025, 9, 1),
    num_days=75
)

display(thermal_v_pack_hotspot_report)

ThermalV Analysis Days: 100%|█████████| 75/75 [01:12<00:00,  1.04it/s]


,Pack ID,Max Temp Duration (min),Min Temp Duration (min)
0,1,18123.82,455020.17
1,2,9373.62,106823.98
2,3,2706.24,16495.67
3,4,1907.29,5676.68
4,5,2467.23,1343.68
5,6,5938.73,1370.54
6,7,12917.42,1002.65
7,8,1317.52,1146.07
8,9,483002.32,1.49
9,10,4037.12,4364.84


In [48]:
thermal_v_pack_hotspot_report.to_csv('hotspot_pack.csv')